In [81]:
import numpy as np
import pandas as pd
from IPython.display import display
import matplotlib.pyplot as plt

In [82]:
train = pd.read_csv("/Users/lramp/Downloads/train_cris.csv")
test = pd.read_csv("/Users/lramp/Downloads/test_cris.csv")

C:\Users\lramp\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:2698: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [83]:
print("Train shape: ",train.shape," Test shape: ",test.shape)

Train shape:  (76020, 371)  Test shape:  (929615, 24)


In [84]:
train_results = train.groupby('TARGET')['ID'].count()
display(train_results)
satisfied = len(train[train['TARGET'] == 1]) / len(train)
print("%1.4f of the observations have good customer satisfaction" % satisfied)

TARGET
0    73012
1     3008
Name: ID, dtype: int64

0.0396 of the observations have good customer satisfaction


In [85]:
target = train['TARGET']
train = train.drop(['TARGET'], axis=1)

# save and remove ID
train_id = train['ID']
train = train.drop('ID', axis = 1)
#test_id = test['ID']
#test = test.drop('ID', axis = 1)

# create an array with both df's for simplicity when cleaning
both_df = [train, test]

In [86]:
# create a function to check nulls
def check_nulls(df):
    nulls = np.sum(df.isnull())
    nullcols = nulls.loc[(nulls != 0)]
    d_types = df.dtypes.loc[(nulls != 0)]
    info = pd.concat([nullcols, d_types], axis=1).sort_values(by=0, ascending=False)
    print(info)
    print("There are", len(nullcols), "columns with missing values")
    
# use the function    
for i in both_df:
    print(check_nulls(i))
    print("------------------------")

Empty DataFrame
Columns: [0, 1]
Index: []
There are 0 columns with missing values
None
------------------------
                     0        1
conyuemp        929511   object
ult_fec_cli_1t  927932   object
cod_prov          3996  float64
nomprov           3996   object
segmento          2248   object
canal_entrada     2081   object
indrel_1mes         23  float64
tiprel_1mes         23   object
sexo                 5   object
There are 9 columns with missing values
None
------------------------


There are no nulls in both train and test.

In [87]:
no_use_train = []
for c in train:
    if train[c].nunique() == 1:
        no_use_train.append(c)

no_use_test = []
for c in test:
    if test[c].nunique() == 1:
        no_use_test.append(c)
        
print("Train: ", len(no_use_train), " Test: ", len(no_use_test))

Train:  34  Test:  2


In [88]:
# test has more so we will drop the extra columns

test = test.drop(no_use_test, axis=1)
train = train.drop(no_use_train, axis=1)

We can see that there are 34 columns in the train df that have only 1 value, while there are 45 columns in the test df that have only 1 value. We will get rid of the higher amount and redo the check to see if there were some in train that were not in test.

### XGB Bayesian optimization

source: https://www.kaggle.com/tilii7/bayesian-optimization-of-xgboost-parameters

In [93]:
# Comment out any parameter you don't want to test
def XGB_CV(
          max_depth,
          gamma,
          min_child_weight,
          max_delta_step,
          subsample,
          colsample_bytree
         ):

    global AUCbest
    global ITERbest

#
# Define all XGboost parameters
#

    paramt = {
              'booster' : 'gbtree',
              'max_depth' : int(max_depth),
              'gamma' : gamma,
              'eta' : 0.05,
              'objective' : 'binary:logistic',
              'nthread' : 4,
              'silent' : True,
              'eval_metric': 'auc',
              'subsample' : max(min(subsample, 1), 0),
              'colsample_bytree' : max(min(colsample_bytree, 1), 0),
              'min_child_weight' : min_child_weight,
              'max_delta_step' : int(max_delta_step),
              'seed' : 65
              }

    folds = 5
    cv_score = 0

    print("\n Search parameters (%d-fold validation):\n %s" % (folds, paramt), file=log_file )
    log_file.flush()

    xgbc = xgb.cv(
                    paramt,
                    dtrain,
                    num_boost_round = 20000,
                    stratified = True,
                    nfold = folds,
#                    verbose_eval = 10,
                    early_stopping_rounds = 100,
                    metrics = 'auc',
                    show_stdv = True
               )

    val_score = xgbc['test-auc-mean'].iloc[-1]
    train_score = xgbc['train-auc-mean'].iloc[-1]
    print(' Stopped after %d iterations with train-auc = %f val-auc = %f ( diff = %f ) train-gini = %f val-gini = %f' % ( len(xgbc), train_score, val_score, (train_score - val_score), (train_score*2-1),
(val_score*2-1)) )
    if ( val_score > AUCbest ):
        AUCbest = val_score
        ITERbest = len(xgbc)

    return (val_score*2) - 1

In [94]:
#Define the log file. If you repeat this run, new output will be added to it
log_file = open('Santander-AUC-5fold-XGB-run-01-v1-full.log', 'a')
AUCbest = -1.
ITERbest = 0


from sklearn.cross_validation import cross_val_score, StratifiedKFold, StratifiedShuffleSplit
import gc
from sklearn.cross_validation import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train, target, test_size=0.2, random_state=69)

In [95]:
dtrain = xgb.DMatrix(train, label = target)

sss = StratifiedShuffleSplit(target, random_state=1001, test_size=0.75)
for train_index, test_index in sss:
    break
#X_train, y_train = train[train_index], target[train_index]
#del train, target
gc.collect()
dtrain = xgb.DMatrix(X_train, label = y_train)

In [96]:
XGB_BO = BayesianOptimization(XGB_CV, {
                                     'max_depth': (2, 12),
                                     'gamma': (0.001, 10.0),
                                     'min_child_weight': (0, 20),
                                     'max_delta_step': (0, 10),
                                     'subsample': (0.4, 1.0),
                                     'colsample_bytree' :(0.4, 1.0)
                                    })

In [97]:
XGB_BO.explore({
              'max_depth':            [3, 8, 3, 8, 8, 3, 8, 3],
              'gamma':                [0.5, 8, 0.2, 9, 0.5, 8, 0.2, 9],
              'min_child_weight':     [0.2, 0.2, 0.2, 0.2, 12, 12, 12, 12],
              'max_delta_step':       [1, 2, 2, 1, 2, 1, 1, 2],
              'subsample':            [0.6, 0.8, 0.6, 0.8, 0.6, 0.8, 0.6, 0.8],
              'colsample_bytree':     [0.6, 0.8, 0.6, 0.8, 0.6, 0.8, 0.6, 0.8],
              })

In [98]:
print('-'*130)
print('-'*130, file=log_file)
log_file.flush()

import warnings
with warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    XGB_BO.maximize(init_points=2, n_iter=5, acq='ei', xi=0.0)

# XGB_BO.maximize(init_points=10, n_iter=50, acq='ei', xi=0.0)
# XGB_BO.maximize(init_points=10, n_iter=50, acq='ei', xi=0.01)
# XGB_BO.maximize(init_points=10, n_iter=50, acq='ucb', kappa=10)
# XGB_BO.maximize(init_points=10, n_iter=50, acq='ucb', kappa=1)

----------------------------------------------------------------------------------------------------------------------------------
Initialization
----------------------------------------------------------------------------------------------------------------------------------
 Step |   Time |      Value |   colsample_bytree |     gamma |   max_delta_step |   max_depth |   min_child_weight |   subsample | 
 Stopped after 246 iterations with train-auc = 0.861983 val-auc = 0.838475 ( diff = 0.023508 ) train-gini = 0.723966 val-gini = 0.676949
    1 | 03m18s |    0.67695 |             0.6000 |    0.5000 |           1.0000 |      3.0000 |             0.2000 |      0.6000 | 
 Stopped after 208 iterations with train-auc = 0.882920 val-auc = 0.838720 ( diff = 0.044199 ) train-gini = 0.765839 val-gini = 0.677441
    2 | 10m22s |    0.67744 |             0.8000 |    8.0000 |           2.0000 |      8.0000 |             0.2000 |      0.8000 | 
 Stopped after 242 iterations with train-auc = 0.8639

In [99]:
print('-'*130)
print('Final Results')
print('Maximum XGBOOST value: %f' % XGB_BO.res['max']['max_val'])
print('Best XGBOOST parameters: ', XGB_BO.res['max']['max_params'])
print('-'*130, file=log_file)
print('Final Result:', file=log_file)
print('Maximum XGBOOST value: %f' % XGB_BO.res['max']['max_val'], file=log_file)
print('Best XGBOOST parameters: ', XGB_BO.res['max']['max_params'], file=log_file)
log_file.flush()
log_file.close()

history_df = pd.DataFrame(XGB_BO.res['all']['params'])
history_df2 = pd.DataFrame(XGB_BO.res['all']['values'])
history_df = pd.concat((history_df, history_df2), axis=1)
history_df.rename(columns = { 0 : 'gini'}, inplace=True)
history_df['AUC'] = ( history_df['gini'] + 1 ) / 2
history_df.to_csv('Santander-AUC-5fold-XGB-run-01-v1-grid.csv')

----------------------------------------------------------------------------------------------------------------------------------
Final Results
Maximum XGBOOST value: 0.678618
Best XGBOOST parameters:  {'max_depth': 4.821237005728406, 'gamma': 1.2173520371228357, 'min_child_weight': 17.48179322089674, 'max_delta_step': 3.3487895352177874, 'subsample': 0.9891855746808983, 'colsample_bytree': 0.4063322759811285}
